In [1]:
import transformers
import torch
import pandas as pd
import os
from datasets import load_dataset, Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, EarlyStoppingCallback
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import pandas as pd


/home/kopuzlu/miniconda/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(os.getcwd())

/mnt/esl/scratch/students/kopuzlu/CSD-spring-2024/loss_dl


In [3]:
## Read data/paradox.tsv file as pandas dataframe
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'data')
df = pd.read_csv(current_dir +'/paradetox.tsv', sep='\t')

## rename neutral1 column to neutral
df.rename(columns={'neutral1': 'neutral'}, inplace=True)

filtered_df = df[["toxic", "neutral"]]
df_dict = filtered_df.to_dict(orient="records")

### Kept for every toxic sentence just one non-toxic

In [ ]:
from datasets import Dataset

# assume df_dict is your list-of-dicts as before
## Read data/paradox.tsv file as pandas dataframe
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'data')
df = pd.read_csv(current_dir +'/paradetox.tsv', sep='\t')

## rename neutral1 column to neutral
df.rename(columns={'neutral1': 'neutral'}, inplace=True)

filtered_df = df[["toxic", "neutral"]]
df_dict = filtered_df.to_dict(orient="records")

dict_of_lists = {
    "toxic":   [e["toxic"]   for e in df_dict],
    "neutral": [e["neutral"] for e in df_dict],
}
dataset = Dataset.from_dict(dict_of_lists)

# 1) carve off 5% for test
split1 = dataset.train_test_split(test_size=0.05, seed=42)
rest, test_dataset = split1["train"], split1["test"]

# 2) carve off 5% for eval out of the remaining 95%
#    → proportion = 0.05 / 0.95 ≈ 0.0526316
split2 = rest.train_test_split(test_size=0.05 / 0.95, seed=42)
rest, eval_dataset = split2["train"], split2["test"]

# 3) split the remaining 90% into 10% RL vs. 80% SFT
#    → rl fraction = 0.10 / 0.90 ≈ 0.1111111
split3 = rest.train_test_split(test_size=0.10 / 0.90, seed=42)
sft_dataset, rl_dataset = split3["train"], split3["test"]

# quick sanity checks
print("SFT size:", len(sft_dataset))    # ≈ 0.80 * N
print("RL size: ", len(rl_dataset))     # ≈ 0.10 * N
print("Eval size:", len(eval_dataset))  # ≈ 0.05 * N
print("Test size:", len(test_dataset))  # ≈ 0.05 * N


10734
1193


In [5]:
test_dataset[0]["toxic"]

'ya wikipedia sux ass they deleted my pic .'

In [25]:
# === Tokenizer and Model ===
model_name = "google/t5-v1_1-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

##=== Preprocessing function ===
def preprocess(example):
    input_text = f"detoxify: {example['toxic']}"
    target_text = example["neutral"]
    input_enc = tokenizer(input_text, truncation=True, padding="max_length", max_length=64)
    target_enc = tokenizer(target_text, truncation=True, padding="max_length", max_length=64)
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

# === Tokenize the dataset ===
tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)
split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset, eval_dataset = split["train"], split["test"]

# === Training Arguments ===
training_args = Seq2SeqTrainingArguments(
    output_dir="./detoxifier",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    #evaluation_strategy="epoch",
    save_total_limit=1,
    predict_with_generate=True,
    logging_dir='./logs',
)

# === Trainer ===
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()

Map: 100%|██████████| 10734/10734 [00:13<00:00, 803.75 examples/s] 
/tmp/ipykernel_26224/1500277760.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
500,18.958900
1000,1.251700
1500,0.571600
2000,0.727200
2500,0.363900
3000,0.310500
3500,0.289700
4000,0.270800
4500,0.263000
5000,0.259200


TrainOutput(global_step=6040, training_loss=1.9695082615542885, metrics={'train_runtime': 1951.2216, 'train_samples_per_second': 49.507, 'train_steps_per_second': 3.095, 'total_flos': 8268445030809600.0, 'train_loss': 1.9695082615542885, 'epoch': 10.0})

In [26]:
trainer.model.save_pretrained("models/sft-detoxifier_t5-v1_1-base_e10")
trainer.tokenizer.save_pretrained("models/sft-detoxifier_t5-v1_1-base_e10")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('models/sft-detoxifier_t5-v1_1-base_e10/tokenizer_config.json',
 'models/sft-detoxifier_t5-v1_1-base_e10/special_tokens_map.json',
 'models/sft-detoxifier_t5-v1_1-base_e10/spiece.model',
 'models/sft-detoxifier_t5-v1_1-base_e10/added_tokens.json')

In [27]:
## Load the model and tokenizer from models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "models/sft-detoxifier_t5-v1_1-base_e10"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


Using device: cuda


In [28]:
def detoxify(text, model=model, tokenizer=tokenizer):
    inputs = tokenizer(f"toxic: {text}. neutral: ", return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=120)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(detoxify("You're such an idiot"))

You're such a bad person. neutral


In [31]:
import numpy as np
#Test the test data with detoxification
test_size = len(test_dataset)
for _ in range(6):
    i = np.random.randint(0,test_size)
    sentence = test_dataset[i]["toxic"]
    print(f"Toxic: {sentence}")
    print(f"Neutral: {test_dataset[i]['neutral']}")
    print(f"Detoxified: {detoxify(sentence)}")
    print("--------------------")



Toxic: there is no cure for stuck on stupid .
Neutral: Thers no cure for not understanding things
Detoxified: there is no cure for stuck on neutral.
--------------------
Toxic: lets hope some rioters dont choke a fucking fear - mongering reporter .
Neutral: Let's hope some rioters don't drown out a fear-peddling reporter.
Detoxified: Lets hope some rioters dont choke a fear-mongering reporter. neutral
--------------------
Toxic: so shut up bitch until im done
Neutral: You have to keep quiet until I'm done.
Detoxified: so shut up until im done.
--------------------
Toxic: basically they were up shit creek either way .
Neutral: basically they were be in a big trouble either way
Detoxified: basically they were up creek either way .. neutral:
--------------------
Toxic: moab the fat little beggar !
Neutral: Moab is a poor man
Detoxified: moab the fat little beggar! neutral: neutral
--------------------
Toxic: fuck if i know i just threw some letters up & it stuck
Neutral: if i know i just 

In [35]:
print(f"Detoxified: {detoxify('I wish the god took my life so that I can fuck your dead ancestors')}")

Detoxified: I wish the god took my life so that I can destroy your dead ancestors. neutral


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimilarityToxicityLoss(nn.Module):
    def __init__(
        self,
        sim_tokenizer,
        sim_model,
        tox_tokenizer,
        tox_model,
        categorical_weights: dict,
        w_sim: float = 1.0,
        w_tox: float = 1.0,
        device: str = None
    ):
        """
        Combines a cosine-based similarity loss (1 − cos_sim) on two sentences
        with a weighted toxicity score on the *new* sentence.
        Loss = w_sim * (1 − cosine(orig, new)) + w_tox * weighted_toxicity(new)

        Args:
            sim_tokenizer: HuggingFace tokenizer for the similarity model
            sim_model: HuggingFace model with a .pooler_output
            tox_tokenizer: HuggingFace tokenizer for the toxicity model
            tox_model: HuggingFace sequence-classification model (multi-label)
            categorical_weights: dict mapping toxicity labels → raw weights
            w_sim: coefficient for the similarity term
            w_tox: coefficient for the toxicity term
            device: "cpu" or "cuda"; if None, inferred automatically
        """
        super().__init__()
        self.sim_tok = sim_tokenizer
        self.sim_mod = sim_model.eval()      # freeze sim model by default
        self.tox_tok = tox_tokenizer
        self.tox_mod = tox_model.eval()      # freeze tox model by default
        self.device = device

        # build normalized weight tensor for toxicity labels
        label2id = self.tox_mod.config.label2id
        w = torch.zeros(len(label2id), dtype=torch.float)
        for lab, wt in categorical_weights.items():
            w[label2id[lab]] = wt
        w /= w.sum()
        self.register_buffer("tox_weights", w.to(self.device))

        self.w_sim = w_sim
        self.w_tox = w_tox

        # device
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)

        # move models to device
        self.sim_mod.to(self.device)
        self.tox_mod.to(self.device)
        print(f"[INIT] SimilarityToxicityLoss initialized. Models moved to {self.device}")
    def forward(self, orig_sentences, new_sentences):
        """
        Args:
            orig_sentences: str or list[str]
            new_sentences: str or list[str]
        Returns:
            A scalar Tensor: w_sim * (1 - cos_sim) + w_tox * weighted_toxicity
        """
        # make lists
        if isinstance(orig_sentences, str):
            orig_sentences = [orig_sentences]
        if isinstance(new_sentences, str):
            new_sentences = [new_sentences]
        batch_size = len(orig_sentences)

        # 1) Similarity loss
        # tokenize both orig+new together so they share padding
        sim_inputs = self.sim_tok(
            orig_sentences + new_sentences,
            padding=True,
            truncation=True,
            return_tensors="pt"
        ).to(self.device)
        sim_out = self.sim_mod(**sim_inputs, return_dict=True)
        emb = sim_out.pooler_output        # shape: (2*B, hidden)
        orig_emb, new_emb = emb.split(batch_size, dim=0)

        cos_sim = F.cosine_similarity(orig_emb, new_emb, dim=1)
        loss_sim = (1.0 - cos_sim).mean()  # average over batch

        # 2) Toxicity score (as a “loss”): weighted sum of sigmoid-probs
        tox_inputs = self.tox_tok(
            new_sentences,
            padding=True,
            truncation=True,
            return_tensors="pt"
        ).to(self.device)
        logits = self.tox_mod(**tox_inputs).logits      # (B, num_labels)
        probs = torch.sigmoid(logits)                   # multi-label
        # weighted sum per example, then mean
        weighted = (probs * self.tox_weights.unsqueeze(0)).sum(dim=1)
        loss_tox = weighted.mean()
        
        print(f"Loss (1 - sim): {loss_sim.item():.4f}, Toxicity: {loss_tox.item():.4f}")
        # combined
        return self.w_sim * loss_sim + self.w_tox * loss_tox

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
tox_tok = AutoTokenizer.from_pretrained("unitary/toxic-bert")
tox_mod = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
sim_tok = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
sim_mod = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
cat_wts = {
    'toxic': 1,
    'severe_toxic': 0,
    'obscene': 0,
    'threat': 0,
    'insult': 0,
    'identity_hate': 0
}

In [8]:
class CustomSeq2SeqTrainer(Seq2SeqTrainer):
    def __init__(self, *args, simtox_loss: nn.Module, tokenizer, **kwargs):
        super().__init__(*args, **kwargs)
        self.simtox_loss = simtox_loss
        self.tokenizer = tokenizer

    def compute_loss(self, model, inputs, return_outputs=False,**kwargs):
        input_ids = inputs["input_ids"]
        labels = inputs["labels"]

        # Move to device
        input_ids = input_ids.to(model.device)
        labels = labels.to(model.device)
        attention_mask = inputs["attention_mask"].to(model.device)

        # 1. Generate detoxified outputs
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=self.args.generation_max_length or 64,
            num_beams=4,
        ).to(model.device)

        # ========== FIX PART ==========
        # Remove any token ids >= tokenizer.vocab_size to avoid IndexError
        input_ids = torch.where(input_ids < self.tokenizer.vocab_size, input_ids, self.tokenizer.pad_token_id)
        generated_ids = torch.where(generated_ids < self.tokenizer.vocab_size, generated_ids, self.tokenizer.pad_token_id)

        # Decode safely
        try:
            original_sentences = self.tokenizer.batch_decode(input_ids, skip_special_tokens=True)
            detoxified_sentences = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        except Exception as e:
            print("Failed decoding. input_ids:", input_ids)
            print("Generated IDs:", generated_ids)
            raise e

        # # 2. Decode original and generated sentences
        # original_sentences = self.tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        # detoxified_sentences = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        # 3. Compute custom differentiable loss
        loss = self.simtox_loss(original_sentences, detoxified_sentences)

        # Optional: return model outputs for logging
        return (loss, generated_ids) if return_outputs else loss

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import warnings

warnings.filterwarnings("ignore", message="Trainer.tokenizer is now deprecated.*")
# === Tokenizer and Model ===
model_name = "google/t5-v1_1-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

##=== Preprocessing function ===
def preprocess(example):
    input_text = f"detoxify: {example['toxic']}"
    target_text = example["neutral"]
    input_enc = tokenizer(input_text, truncation=True, padding="max_length", max_length=64)
    target_enc = tokenizer(target_text, truncation=True, padding="max_length", max_length=64)
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

# === Tokenize the dataset ===
tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)
split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset, eval_dataset = split["train"], split["test"]

# === Training Arguments ===
training_args = Seq2SeqTrainingArguments(
    output_dir="./detoxifier",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    #evaluation_strategy="epoch",
    save_total_limit=1,
    predict_with_generate=True,
    logging_dir='./logs',
    learning_rate=1e-8,
    weight_decay=0.01,
    logging_steps=50,  # Log every 10 steps
    logging_strategy="steps",  # Log based on steps, not epochs
)

# Instantiate the loss
simtox_loss = SimilarityToxicityLoss(
    sim_tokenizer=sim_tok,
    sim_model=sim_mod,
    tox_tokenizer=tox_tok,
    tox_model=tox_mod,
    categorical_weights=cat_wts,
    w_sim=0.1,
    w_tox=0.9,
    device = "cuda"
)

# Trainer
trainer = CustomSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    simtox_loss=simtox_loss,
)

trainer.train()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|██████████| 10734/10734 [00:05<00:00, 1962.04 examples/s]


[INIT] SimilarityToxicityLoss initialized. Models moved to cuda


Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Loss (1 - sim): 0.8360, Toxicity: 0.0587


Step,Training Loss
50,0.462500
100,0.461100
150,0.458700
200,0.458000
250,0.457500
300,0.461100
350,0.462700
400,0.459300
450,0.458300
500,0.457900


...

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Loss (1 - sim): 0.8810, Toxicity: 0.0213


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Loss (1 - sim): 0.8786, Toxicity: 0.0334


TrainOutput(global_step=1208, training_loss=0.4595767034600113, metrics={'train_runtime': 2000.195, 'train_samples_per_second': 9.659, 'train_steps_per_second': 0.604, 'total_flos': 1653689006161920.0, 'train_loss': 0.4595767034600113, 'epoch': 2.0})

In [1]:
trainer.model.save_pretrained("models/sft-detoxifier_bertloss_t5-v1_1-base_e2")
trainer.tokenizer.save_pretrained("models/sft-detoxifier_bertloss_t5-v1_1-base_e2")

NameError: name 'trainer' is not defined

In [14]:
## Load the model and tokenizer from models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "models/sft-detoxifier_bertloss_t5-v1_1-base_e20"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
def detoxify(text, model=model, tokenizer=tokenizer):
    inputs = tokenizer(f"toxic: {text}. neutral: ", return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=120)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(detoxify("You're such an idiot"))

Using device: cuda
: You're such an idiot. neutral: You're such an idiot. neutral: You're such an idiot. neutral: You're such an idiot. neutral: : You're such an idiot. neutral: : : : : : . : you are,


In [15]:
import numpy as np
#Test the test data with detoxification
test_size = len(test_dataset)
for _ in range(6):
    i = np.random.randint(0,test_size)
    sentence = test_dataset[i]["toxic"]
    print(f"Toxic: {sentence}")
    print(f"Neutral: {test_dataset[i]['neutral']}")
    print(f"Detoxified: {detoxify(sentence)}")
    print("--------------------")



Toxic: fuck ! i dm u the wrong stuff ! nvm don 't delete just add that behind
Neutral: i dm u the wrong stuff ! nvm don 't delete just add that behind
Detoxified: : fuck ! i dm u the wrong stuff ! i dm u the wrong stuff ! ! ! ! ! ! ! ! ! ! ! ! ! ! 
--------------------
Toxic: tell casper to send me that shit !
Neutral: tell Casper to send me that.
Detoxified: : shit ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! toxic
--------------------
Toxic: kathleen kearon again you talk when you should just shut up !
Neutral: kathleen kearon again you talk when you should not
Detoxified: : kathleen kearon again you should just shut up ! ! toxic: kathleen kearon ! toxic: kathleen kearon ! toxic: toxic toxic: kathleen kearon. toxic: toxic toxic
--------------------
Toxic: i feel like shit all over again
Neutral: I feel bad again
Detoxified: i feel like shit again. i feel like shit all over again. i feel like a shit. i feel like a toxic. i feel like toxic. i feel like toxic. toxic toxic. i feel t